In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dropout

**ПРЕДОБРАБОТКА**

In [ ]:
data_dir = 'data'
categories = ['black_white', 'color']

img_size = 64

def load_data():
    images = []
    labels = []
    for category in categories:
        path = os.path.join(data_dir, category)
        class_num = categories.index(category)  # 0 для bl_white и 1 для color
        for img_name in os.listdir(path):
            try:
                img_path = os.path.join(path, img_name)
                img = load_img(img_path, target_size=(img_size, img_size))
                img_array = img_to_array(img)
                images.append(img_array)
                labels.append(class_num)
            except Exception as e:
                print(f"Ошибка загрузки изображения {img_name}: {e}")
    return np.array(images), np.array(labels)

X, y = load_data()
X = X / 255.0

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

**МОДЕЛЬ**

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping

datagen = ImageDataGenerator(
    rotation_range=15,   
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = datagen.flow(X_train, y_train, batch_size=64)
validation_generator = datagen.flow(X_test, y_test, batch_size=64, shuffle=False)

model = Sequential([
    BatchNormalization(),
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    
    Dropout(0.5),
    
    Conv2D(128, (3, 3), activation='relu'),
    
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])


early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(
    train_generator,                      
    steps_per_epoch=len(X_train) // 32,  
    validation_data=(X_test, y_test),    
    epochs=80,                            
    callbacks=[early_stopping]           
)


**ДООБУЧЕНИЕ В СЛУЧАЕ НАДОБНОСТИ НА ОСНОВЕ УЖЕ ПОЛУЧЕННЫХ ВЕСОВ**

In [ ]:
model.save_weights("model_weights.weights.h5")
model.load_weights("model_weights.weights.h5")

history_finetune = model.fit(
    train_generator,  
    steps_per_epoch=len(X_train) // 32,  
    validation_data=validation_generator,  
    epochs=20,  
    callbacks=[early_stopping]
)


**ПОСТРОЕНИЕ ГРАФИКА ОБУЧЕНИЯ. ЭПОХИ/ТОЧНОСТЬ**

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(history.history['accuracy'], label='Точность на обучении', color='blue', linestyle='-', marker='o')
plt.plot(history.history['val_accuracy'], label='Точность на валидации', color='orange', linestyle='--', marker='x')
plt.title('Точность модели на обучении и валидации', fontsize=16)
plt.xlabel('Эпоха', fontsize=12)
plt.ylabel('Точность', fontsize=12)
plt.legend(loc='lower right', fontsize=12)
plt.grid(True)
plt.tight_layout()
plt.show()

**КЛАССИФИКАЦИЯ**

In [ ]:
import os
import shutil
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
input_dir = 'unclassified_images'
output_dir = 'classified_images'
def classify_and_copy_images(input_dir, output_dir, model, img_size=64):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for img_name in os.listdir(input_dir):
        img_path = os.path.join(input_dir, img_name)
        img = load_img(img_path, target_size=(img_size, img_size))
        img_array = img_to_array(img) / 255.0
        img_array = np.expand_dims(img_array, axis=0)
        prediction = model.predict(img_array)[0][0]  # Берём первый элемент массива
        category = 'color' if prediction > 0.9 else 'black_white'
        new_name = f"{os.path.splitext(img_name)[0]}_{prediction:.4f}.jpg"

        dest_dir = os.path.join(output_dir, category)
        if not os.path.exists(dest_dir):
            os.makedirs(dest_dir)

        new_path = os.path.join(dest_dir, new_name)

        shutil.copy2(img_path, new_path)
        print(f"Копия {img_name} → {category}/{new_name} (Вероятность: {prediction:.4f})")


**ЗАПУСК ФУНКЦИИ ОБРАБОТКИ ИЗОБРАЖЕНИЙ**

In [ ]:
classify_and_copy_images("unclassified_images", "classified_images", model)